<a href="https://colab.research.google.com/github/YXLIU926/Walmart-M5-Sales-Prediction/blob/main/M5_Forecasting_Walmart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective

* Make a baseline model that predict the validation (28 days). 
* This competition has 2 stages, so the main objective is to make a model that can predict the demand for the next 28 days

In [ ]:
!pip install 'fsspec>=0.3.3'
!pip install 'dask_xgboost'

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import dask.dataframe as dd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import dask_xgboost as xgb
import dask.dataframe as dd
from sklearn import preprocessing, metrics
import gc
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# function to read the data and merge it (ignoring some columns, this is a very fst model)
def read_data():
    print('Reading files...')
    calendar = pd.read_csv('/content/drive/Shareddrives/HW2 Shared Drive/calendar.csv')
    calendar1 = pd.read_csv('/content/drive/Shareddrives/HW2 Shared Drive/calendar.csv')
    calendar2 = pd.read_csv('/content/drive/Shareddrives/HW2 Shared Drive/calendar.csv')
    calendar3 = pd.read_csv('/content/drive/Shareddrives/HW2 Shared Drive/calendar.csv')
    #calendar = reduce_mem_usage(calendar)
    print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))
    sell_prices = pd.read_csv('/content/drive/Shareddrives/HW2 Shared Drive/sell_prices.csv')
    #sell_prices = reduce_mem_usage(sell_prices)
    print('Sell prices has {} rows and {} columns'.format(sell_prices.shape[0], sell_prices.shape[1]))
    sales_train_validation = pd.read_csv('/content/drive/Shareddrives/HW2 Shared Drive/sales_train_validation.csv')
    print('Sales train evaluation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    submission = pd.read_csv('/content/drive/Shareddrives/HW2 Shared Drive/sample_submission.csv')
    
    return calendar, calendar1, calendar2, calendar3, sell_prices, sales_train_validation,submission

calendar, calendar1, calendar2, calendar3, sell_prices, sales_train_validation, submission = read_data()

# MEAN OF EACH ROW
meanDataframe = sales_train_validation.mean(axis=1)

In [ ]:
# NORMALIZATION
# for row in sales_train_validation:
prefix = "d_"
count = 1
lstFeatures = []
lstFeaturesSTD = []
while (count <= 1913):
  columnName = prefix + str(count)
  lstFeatures.append(columnName)
  count += 1
# print(lstFeatures)
sales_train_validation = sales_train_validation[lstFeatures]
sales_train_validation = sales_train_validation.apply(lambda x: x/x.max(), axis = 1)
sales_train_validation = sales_train_validation.fillna(0)
print(sales_train_validation)

# for i in range(len(sales_train_validation)):
#   for j in range(6, len(sales_train_validation.columns), 1):
#     # print(sales_train_validation.iat[i,j])
#     # sales_train_validation.iat[i,j] = sales_train_validation.iat[i,j] - meanDataframe.iat[i]
#     print(sales_train_validation.iat[i,j])
#     print(meanDataframe.iat[i])
#     print(sales_train_validation.iat[i,j] - meanDataframe.iat[i])
    # print(sales_train_validation.iat[i,j])
# print(sales_train_validation)
# print(meanDataframe)
# print(meanDataframe.mean(axis=1))

# print()
#   break

In [ ]:
# Find 20 days (D1-D1913) that have the largest variances in terms of item sales (Standard deviations), and use these as the features for further clustering

prefix = "d_"
count = 1
lstFeaturesSTD = []
while (count <= 1913):
  columnName = prefix + str(count)
  lstFeaturesSTD.append(sales_train_validation[columnName].std())
  count += 1

indexList = sorted(range(len(lstFeaturesSTD)), key=lambda x: lstFeaturesSTD[x])[-20:]
nameList = [prefix + str(x + 1) for x in indexList]

clusterNumberDF = sales_train_validation[nameList]
# clusterNumberDF = clusterNumberDF.fillna(0)
print(clusterNumberDF)


In [ ]:
#Detect if there's any NA value
clusterNumberDF.isnull().values.any()

In [ ]:
# FIND OPTIMAL CLUSTER NUMBER
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt


distortions = []
inertias = []
mapping1 = {}
mapping2 = {}
K = range(1,20)
X = clusterNumberDF.to_numpy()

print(np.argwhere(np.isnan(X)))

for k in K:
    # Building and fitting the model
    kmeanModel = KMeans(n_clusters=k).fit(X)
    kmeanModel.fit(X)
 
    distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_,
                                        'euclidean'), axis=1)) / X.shape[0])
    inertias.append(kmeanModel.inertia_)
 
    mapping1[k] = sum(np.min(cdist(X, kmeanModel.cluster_centers_,
                                   'euclidean'), axis=1)) / X.shape[0]
    mapping2[k] = kmeanModel.inertia_

In [ ]:
plt.plot(K, distortions, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Distortion')
plt.title('The Elbow Method using Distortion')
plt.show()

In [ ]:
X = clusterNumberDF.to_numpy()

In [ ]:
# CLUSTER
from sklearn.cluster import KMeans
kmeans = KMeans(3, random_state = 42)
kmeans.fit(X)
identified_clusters = kmeans.fit_predict(X)

In [ ]:
# DATA DIMENSION
print(len(identified_clusters))
print(max(identified_clusters))

In [ ]:
sales_train_validation = pd.read_csv('/content/drive/Shareddrives/HW2 Shared Drive/sales_train_validation.csv')
print('Sales train evaluation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))

In [ ]:
# ADD CLUSTER ID TO ORIGINAL DATA
clusterIdList = list(identified_clusters)
sales_train_validation['clusterid'] = clusterIdList

In [ ]:
# TEST CLUSTER 1
cluster1 = sales_train_validation[(sales_train_validation.clusterid == 0)]
del cluster1["clusterid"]
print(cluster1)

In [ ]:
# TEST CLUSTER 2
cluster2 = sales_train_validation[(sales_train_validation.clusterid == 1)]
del cluster2["clusterid"]
print(cluster2)

In [ ]:
# CLUSTER 3
cluster3 = sales_train_validation[(sales_train_validation.clusterid == 2)]
del cluster3["clusterid"]
print(cluster3)

In [ ]:
def melt_and_merge(calendar, sell_prices, cluster, submission, merge = False):
    
    # melt sales data, get it ready for training
    cluster = pd.melt(cluster, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    print('Melted sales train validation has {} rows and {} columns'.format(cluster.shape[0], cluster.shape[1]))
    #cluster = reduce_mem_usage(cluster)
    
    # seperate test dataframes
    test1_rows = [row for row in submission['id'] if 'validation' in row]
    test2_rows = [row for row in submission['id'] if 'evaluation' in row]
    test1 = submission[submission['id'].isin(test1_rows)]
    test2 = submission[submission['id'].isin(test2_rows)]
    
    # change column names
    test1.columns = ['id', 'd_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919', 'd_1920', 'd_1921', 'd_1922', 'd_1923', 'd_1924', 'd_1925', 'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 'd_1931', 
                      'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938', 'd_1939', 'd_1940', 'd_1941']
    test2.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949', 'd_1950', 'd_1951', 'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958', 'd_1959', 
                      'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967', 'd_1968', 'd_1969']
    
    # get product table
    product = cluster[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()
    
    # merge with product table
    test2['id'] = test2['id'].str.replace('_evaluation','_validation')
    test1 = test1.merge(product, how = 'left', on = 'id')
    test2 = test2.merge(product, how = 'left', on = 'id')
    test2['id'] = test2['id'].str.replace('_validation','_evaluation')
    
    # 
    test1 = pd.melt(test1, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    test2 = pd.melt(test2, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    
    cluster['part'] = 'train'
    test1['part'] = 'test1'
    test2['part'] = 'test2'
    
    data = pd.concat([cluster, test1, test2], axis = 0)
    
    del cluster, test1, test2
    
    # get only a sample for fst training
    #data = data.loc[nrows:]
    
    # drop some calendar features
    calendar.drop(['weekday', 'wday', 'month', 'year'], inplace = True, axis = 1)
    
    # delete test2 for now
    data = data[data['part'] != 'test2']
    
    if merge:
        # notebook crash with the entire dataset (maybee use tensorflow, dask, pyspark xD)
        data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
        data.drop(['d', 'day'], inplace = True, axis = 1)
        # get the sell price data (this feature should be very important)
        data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
        print('Our final dataset to train has {} rows and {} columns'.format(data.shape[0], data.shape[1]))
    else: 
        pass
    
    gc.collect()
    
    return data
        
#calendar, sell_prices, cluster, submission = read_data()
#data = melt_and_merge(calendar, sell_prices, cluster, submission,  merge = True)

In [ ]:
#Total
#data_total = melt_and_merge(calendar, sell_prices, sales_train_validation, submission, merge = True)
#data_total

In [ ]:
#Cluster 1 data
data1 = melt_and_merge(calendar1, sell_prices, cluster1, submission, merge = True)


In [ ]:
data1

In [ ]:
data2 = melt_and_merge(calendar2, sell_prices, cluster2, submission, merge = True)

In [ ]:
data3 = melt_and_merge(calendar3, sell_prices, cluster3, submission, merge = True)

In [ ]:
#Look at each cluster's characteristics
#Cluster 1
data1.describe(percentiles=None, include=None, exclude=None, datetime_is_numeric=False)
data1.astype('object').describe().transpose()

In [ ]:
#Cluster 2
data2.describe(percentiles=None, include=None, exclude=None, datetime_is_numeric=False)
data2.astype('object').describe().transpose()

In [ ]:
#Cluster 3
data3.describe(percentiles=None, include=None, exclude=None, datetime_is_numeric=False)
data3.astype('object').describe().transpose()

In [ ]:
print(sales_train_validation)
# calendar, sell_prices, sales_train_validation, submission = read_data()

* We have the data to build our first model, let's build a baseline and predict the validation data (in our case is test1)

In [ ]:
def transform(data):
    nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in nan_features:
        data[feature].fillna('unknown', inplace = True)    
    cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in cat:
        encoder = preprocessing.LabelEncoder()
        data[feature] = encoder.fit_transform(data[feature])
    return data

def simple_fe(data):  
    # rolling demand features
    data['lag_t28'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28))
    data['lag_t29'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(29))
    data['lag_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(30))
    data['rolling_mean_t7'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).mean())
    data['rolling_std_t7'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).std())
    data['rolling_mean_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).mean())
    data['rolling_mean_t90'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(90).mean())
    data['rolling_mean_t180'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(180).mean())
    data['rolling_std_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).std())
    data['rolling_skew_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).skew())
    data['rolling_kurt_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).kurt())  
    # price features
    data['lag_price_t1'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1))
    data['price_change_t1'] = (data['lag_price_t1'] - data['sell_price']) / (data['lag_price_t1'])
    data['rolling_price_max_t365'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())
    data['price_change_t365'] = (data['rolling_price_max_t365'] - data['sell_price']) / (data['rolling_price_max_t365'])
    data['rolling_price_std_t7'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(7).std())
    data['rolling_price_std_t30'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(30).std())
    data.drop(['rolling_price_max_t365', 'lag_price_t1'], inplace = True, axis = 1)
    # time features
    data['date'] = pd.to_datetime(data['date'])
    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    data['week'] = data['date'].dt.week
    data['day'] = data['date'].dt.day
    data['dayofweek'] = data['date'].dt.dayofweek
    
    return data

def run_lgb(data):
    # going to evaluate with the last 28 days
    x_train = data[data['date'] <= '2016-03-27']
    y_train = x_train['demand']
    x_val = data[(data['date'] > '2016-03-27') & (data['date'] <= '2016-04-24')]
    y_val = x_val['demand']
    test = data[(data['date'] > '2016-04-24')]
    del data
    gc.collect()
    # hyperparameter tunning 
    # define random hyperparammeters
    params = {
        'boosting_type': 'gbdt',
        'metric': 'rmse',
        'objective': 'regression',
        'n_jobs': -1,
        'seed': 236,
        'learning_rate': 0.1,
        'bagging_fraction': 0.75,
        'bagging_freq': 10, 
        'colsample_bytree': 0.75}

    train_set = lgb.Dataset(x_train[features], y_train)
    val_set = lgb.Dataset(x_val[features], y_val)
    del x_train, y_train

    model = lgb.train(params, train_set, num_boost_round = 2500, early_stopping_rounds = 50, valid_sets = [train_set, val_set], verbose_eval = 100)
    val_pred = model.predict(x_val[features])
    val_score = np.sqrt(metrics.mean_squared_error(val_pred, y_val))
    print(f'Our val rmse score is {val_score}')
    y_pred = model.predict(test[features])
    test['demand'] = y_pred
    return test
def predict(test, submission):
    predictions = test[['id', 'date', 'demand']]
    predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'demand').reset_index()
    predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

    evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
    evaluation = submission[submission['id'].isin(evaluation_rows)]

    validation = submission[['id']].merge(predictions, on = 'id')
    final = pd.concat([validation, evaluation])
    final.to_csv('submission.csv', index = False)

# define list of features
features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'year', 'month', 'week', 'day', 'dayofweek', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 
            'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_t28', 'lag_t29', 'lag_t30', 'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t30', 'rolling_mean_t90', 
            'rolling_mean_t180', 'rolling_std_t30', 'price_change_t1', 'price_change_t365', 'rolling_price_std_t7', 'rolling_price_std_t30', 'rolling_skew_t30', 'rolling_kurt_t30']


In [ ]:
def transform_train_and_eval(data):
    data = transform(data)
    data = simple_fe(data)
    # reduce memory for new features so we can train
    #data = reduce_mem_usage(data)
    test = run_lgb(data)
    predict(test, submission)

In [ ]:
# print(data3)
# print(data3.groupby(['id'])['demand'].transform(lambda x: x.shift(28)))
# print(data3)
# print(data3['id'] == 'HOBBIES_1_029_CA_1_validation')
# print(data3.loc[data3['id'] == 'HOBBIES_1_029_CA_1_validation')
# for i in data3.groupby(['id'])['demand']:
#   print(i)

In [ ]:
#The actual data transform and simplification, along with predicting stage.
#I start with cludter 3 cuz it's the smallest cluster along all three and thereby has the shortest runtime
data3 = transform(data3)
data3 = simple_fe(data3)
print(data3)

In [ ]:
test = run_lgb(data3)
predict(test, submission)
cluster3_csv = test.to_csv('submission_cluster3.csv')

In [ ]:
#After confirming cluster 3 worked, moving onto cluster 1 and 2

In [ ]:
cluster1_predict = transform_train_and_eval(data1)
cluster1_csv = test.to_csv('submission_cluster1.csv')

In [ ]:
cluster2_predict = transform_train_and_eval(data2)
cluster2_csv = test.to_csv('submission_cluster2.csv')